link model: https://drive.google.com/drive/folders/1Uba4fZReIqVw_3hFFCr2hMxC_tmHUfxZ?usp=sharing  
link data train đã chia sẵn: https://drive.google.com/drive/folders/1Uba4fZReIqVw_3hFFCr2hMxC_tmHUfxZ?usp=sharing

## Conect google driver

In [1]:
from google.colab import drive
import pandas as pd
drive.mount('/content/drive', force_remount=True)
model_name = '/content/drive/My Drive/ura-llama-7b'
train_file = '/content/drive/My Drive/train.csv'
new_model = "llama-2-7b-new"

Mounted at /content/drive


In [2]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 52.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 47.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 36.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.2 MB/s eta 0:00:00


## Import library

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset
from tqdm import tqdm
import torch
from datasets import Dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer


'NoneType' object has no attribute 'cadam32bit_grad_fp32'


c:\Users\ADMIN\AppData\Local\Programs\Python\Python39\lib\site-packages\bitsandbytes\cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


## Read data

In [ ]:
train_file = "./data/train.csv"
df = pd.read_csv(train_file)
columns_to_keep = ['description', 'label']
df = df[columns_to_keep]
df.head()

,description,label
0,"First, I want to point out that just about eve...",NEG
1,"Before ""The Birth of a Nation,"" motion picture...",POS
2,...yet I still give it an 8/10 for all of the ...,POS
3,I've read a few of the 10's at this site and a...,NEG
4,A lot of people see this film as an artistic t...,NEG


## Processing data

In [ ]:
def generate_prompt(data_point):
    return f"""
            Analyze the sentiment of the news headline enclosed in square brackets, determine if it is positive, neutral, or negative, and return the answer as the corresponding sentiment label "positive" or "neutral" or "negative".[{data_point["description"]}] = {data_point["label"]}
            """.strip()

def generate_test_prompt(data_point):
    return f"""
            Analyze the sentiment of the news headline enclosed in square brackets, determine if it is positive, neutral, or negative, and return the answer as the corresponding sentiment label "positive" or "neutral" or "negative".[{data_point["description"]}] =
            """.strip()


In [ ]:
train, test = train_test_split(df, test_size= 0.2, random_state= 42)
train_data = train.apply(generate_prompt, axis=1)
df_train = pd.DataFrame(
    {'text':train_data}
)
test_data = test.apply(generate_test_prompt, axis=1)
df_test = pd.DataFrame(
    {'text':test_data.iloc[:100]}
)
y_true = test['label']
train_data = Dataset.from_pandas(df_train)
test_data = Dataset.from_pandas(df_test)

In [3]:
pd.DataFrame(
    {'text':train_data}
)

NameError: name 'train_data' is not defined

## define QLoRA parameters

In [6]:


################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 64 #16

# Dropout probability for LoRA layers
lora_dropout = 0.05 #0.1

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 1

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 4

# Batch size per GPU for evaluation
per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.2

# Initial learning rate (AdamW optimizer)
learning_rate = 3e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule
lr_scheduler_type = "cosine"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 0

# Log every X updates steps
logging_steps = 25

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)


bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit ,
    bnb_4bit_quant_type=bnb_4bit_quant_type ,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

AssertionError: Torch not compiled with CUDA enabled

In [1]:
import torch
torch.cuda.is_available()

True

In [10]:
#Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)


# Train model
trainer.train()

# Save trained model
trainer.model.save_pretrained(new_model)

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/255357 [00:00<?, ? examples/s]

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
25,2.559800
50,2.838200
75,2.484300
100,2.142700
125,2.218700
150,1.568900
175,2.083900
200,1.667100


KeyboardInterrupt: ignored

In [32]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)


def predict(test, model, tokenizer):
    y_pred = []
    for i in tqdm(range(len(df_test))):
        prompt = df_test.iloc[i]["text"]
        pipe = pipeline(task="text-generation",
                        model=model,
                        tokenizer=tokenizer,
                        max_new_tokens = 2,
                        temperature = 0.0,
                       )
        result = pipe(prompt)
        answer = result[0]['generated_text'].split("=")[-1].strip()
        if "POS" in answer:
            y_pred.append("POS")
        elif "NEG" in answer:
            y_pred.append("NEG")
        elif "NEU" in answer:
            y_pred.append("NEU")
        else:
            y_pred.append("none")
    return y_pred


y_pred = predict(test, model, tokenizer)

100%|██████████| 100/100 [02:09<00:00,  1.29s/it]


In [30]:
prompt = df_test.iloc[0]["text"]
pipe = pipeline(task="text-generation",
                model=model,
                tokenizer=tokenizer,
                max_new_tokens = 2,
                temperature = 0.0,
                )
result = pipe(prompt)
result[0]['generated_text'].split("=")[-1].strip()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


'POS'

In [34]:
dict_res = {
    'len': 0,
    'POS': 0,
    'NEG': 0,
    'NEU': 0
}

for i in range(100):
    dict_res['len']+=1
    if y_true.iloc[i] == y_pred[i]:
        dict_res[y_true.iloc[i]] +=1
dict_res

{'len': 100, 'POS': 76, 'NEG': 0, 'NEU': 5}

In [33]:
y_pred

['POS',
 'POS',
 'POS',
 'POS',
 'POS',
 'POS',
 'POS',
 'POS',
 'POS',
 'POS',
 'NEU',
 'NEU',
 'POS',
 'NEU',
 'POS',
 'POS',
 'POS',
 'POS',
 'POS',
 'POS',
 'POS',
 'POS',
 'POS',
 'POS',
 'POS',
 'POS',
 'POS',
 'POS',
 'POS',
 'POS',
 'POS',
 'POS',
 'POS',
 'POS',
 'POS',
 'POS',
 'POS',
 'POS',
 'POS',
 'POS',
 'POS',
 'POS',
 'POS',
 'POS',
 'POS',
 'POS',
 'POS',
 'POS',
 'POS',
 'POS',
 'NEU',
 'POS',
 'POS',
 'POS',
 'POS',
 'POS',
 'POS',
 'NEU',
 'NEU',
 'POS',
 'POS',
 'POS',
 'POS',
 'POS',
 'POS',
 'NEU',
 'POS',
 'POS',
 'POS',
 'POS',
 'POS',
 'NEU',
 'POS',
 'POS',
 'NEU',
 'POS',
 'NEG',
 'POS',
 'POS',
 'POS',
 'NEU',
 'POS',
 'POS',
 'POS',
 'POS',
 'POS',
 'POS',
 'POS',
 'POS',
 'POS',
 'POS',
 'POS',
 'POS',
 'POS',
 'NEU',
 'none',
 'POS',
 'POS',
 'none',
 'POS']

In [ ]:
['Documentary', 'Short']

In [3]:
import spacy
import numpy as np

# Load the pre-trained spaCy model with word vectors
nlp = spacy.load("en_core_web_md")

def semantic_search(query, text_list, threshold=0.5):
    query_embedding = nlp(query).vector

    results = []
    for i, text in enumerate(text_list):
        text_embedding = nlp(text).vector
        similarity = np.dot(query_embedding, text_embedding) / (np.linalg.norm(query_embedding) * np.linalg.norm(text_embedding))
        if similarity >= threshold:
            results.append((text, similarity))

    return results

# Example usage:
list_of_texts = ["This is the first text.", "Here's the second text.", "And the third text."]

input_text_to_search = "second"

result_indices = semantic_search(input_text_to_search, list_of_texts)

if result_indices:
    print(f"Input text '{input_text_to_search}' found in texts with similarities: {result_indices}")
else:
    print(f"Input text '{input_text_to_search}' not found in any texts.")


Input text 'second' found in texts with similarities: [("Here's the second text.", 0.55008525), ('And the third text.', 0.5579095)]
